# Pipeline for High-z Radio Galaxies 12: Create lists (files) with radio AGN candidates

## Introduction

In this file, three models will be applied consecutively in order to predict  
the detection of Radio Galaxies (radio AGN) and their redshift.  

In principle, this pipeline should be applied to data in Stripe 82. But  
it can be used with any other suitable dataset.

In [1]:
%matplotlib inline
# Static plots
#%matplotlib ipympl
# Interactive plots
import numpy as np
import matplotlib as mpl
import matplotlib.cm as cm
from matplotlib import ticker
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.patheffects as mpe
import matplotlib.patches as mpatches
from matplotlib.ticker import ScalarFormatter
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from astropy.visualization import LogStretch, PowerStretch
from astropy.visualization.mpl_normalize import ImageNormalize
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import ConfusionMatrixDisplay
import sklearn.pipeline
import colorcet as cc
from pycaret import classification as pyc
from pycaret import regression as pyr
from joblib import dump, load
import pandas as pd
import mpl_scatter_density
import global_variables as gv
import global_functions as gf

In [2]:
# mpl.rcdefaults()

#### Methods to predict values using individual models

Predict AGN/Galaxy classification

In [3]:
def predict_AGN_gal(catalog_df, 
                    AGN_gal_model, 
                    cal_AGN_gal_model, 
                    threshold, 
                    cal_threshold, 
                    raw_score=True):
    catalog_df = pyc.predict_model(AGN_gal_model, 
                                   data=catalog_df, 
                                   probability_threshold=threshold, 
                                   raw_score=raw_score, 
                                   round=10)
    catalog_df = catalog_df.drop(columns=['Score_0'])
    catalog_df = catalog_df.rename(columns={'Label': 'pred_class', 'Score_1': 'Score_AGN'})
    catalog_df.loc[:, 'Score_AGN'] = np.around(catalog_df.loc[:, 'Score_AGN'], decimals=8)
    pred_probs = cal_AGN_gal_model.predict(catalog_df.loc[:, 'Score_AGN'])
    cal_class  = np.array(pred_probs >= cal_threshold).astype(int)
    catalog_df['Prob_AGN']       = pred_probs
    catalog_df['pred_class_cal'] = cal_class
    return catalog_df

Predict radio detection for AGN

In [4]:
def predict_radio_det(catalog_df, 
                      radio_model, 
                      cal_radio_model, 
                      threshold, 
                      cal_threshold, 
                      raw_score=True):
    catalog_df = pyc.predict_model(radio_model, 
                                   data=catalog_df, 
                                   probability_threshold=threshold, 
                                   raw_score=raw_score, 
                                   round=10)
    catalog_df = catalog_df.drop(columns=['Score_0'])
    catalog_df = catalog_df.rename(columns={'Label': 'pred_radio', 'Score_1': 'Score_radio'})
    catalog_df.loc[:, 'Score_radio'] = np.around(catalog_df.loc[:, 'Score_radio'], decimals=8)
    pred_probs = cal_radio_model.predict(catalog_df.loc[:, 'Score_radio'])
    cal_class  = np.array(pred_probs >= cal_threshold).astype(int)
    catalog_df['Prob_radio']     = pred_probs
    catalog_df['pred_radio_cal'] = cal_class
    return catalog_df

Predict redshift for radio-detected AGN

In [5]:
def predict_z(catalog_df, 
              redshift_model):
    catalog_df = pyr.predict_model(redshift_model, 
                                   data=catalog_df, 
                                   round=10)
    catalog_df = catalog_df.rename(columns={'Label': 'pred_Z'})
    catalog_df.loc[:, 'pred_Z'] = np.around(catalog_df.loc[:, 'pred_Z'], decimals=4)
    return catalog_df

---

## Reading data

Flags.

In [6]:
save_plot_flag      = False
save_preds_flag     = True
load_models_flag    = False
predict_only_hi_z   = False
use_zeroth_model    = False
use_second_z_model  = False  # z >= 3.6 (with SMOGN), or, if needed, z >= 4.0

In [7]:
used_area           = 'HETDEX'  # can be 'S82', 'HETDEX', 'COSMOS'

In [8]:
file_name_dict      = {'S82': gv.file_S82, 'HETDEX': gv.file_HETDEX, 'COSMOS': gv.file_COSMOS}
file_name           = file_name_dict[used_area]

In [9]:
feats_2_disc_S82    = ['objID', 'RA_MILLI', 'DEC_MILLI', 'W1mag', 'W2mag', 'num_imputed', 'radio_detect']
feats_2_disc_HETDEX = ['objID', 'RA_MILLI', 'DEC_MILLI', 'W1mag', 'W2mag', 'num_imputed', 'radio_detect']
feats_2_disc_COSMOS = ['objID', 'RA_MILLI', 'DEC_MILLI', 'W1mag', 'W2mag', 'num_imputed', 'radio_detect', ]

feats_2_disc        = {'S82': feats_2_disc_S82, 'HETDEX': feats_2_disc_HETDEX, 'COSMOS': feats_2_disc_COSMOS}
features_2_discard  = feats_2_disc[used_area]

In [10]:
full_catalog_df     = pd.read_hdf(gv.cat_path + file_name, key='df').drop(columns=features_2_discard)

In [11]:
if used_area == 'S82':
    full_catalog_df.loc[:, 'LOFAR_detect'] = full_catalog_df.loc[:, 'VLAS82_detect'].copy()
    full_catalog_df = full_catalog_df.drop(columns=['VLAS82_detect'])
if used_area == 'COSMOS':
    full_catalog_df.loc[:, 'LOFAR_detect'] = full_catalog_df.loc[:, 'COSMOSVLA3_detect'].copy()
    full_catalog_df = full_catalog_df.drop(columns=['COSMOSVLA3_detect'])

Create features with class and combined redshift.

In [12]:
full_catalog_df['class']            = full_catalog_df.loc[:, 'is_AGN'].copy()
filter_non_confirmed                = np.array(full_catalog_df.loc[:, 'is_AGN'] == 1) | np.array(full_catalog_df.loc[:, 'is_gal'] == 1)
full_catalog_df.loc[~filter_non_confirmed, 'class'] = np.nan
idx_non_Z                           = full_catalog_df.loc[:, 'Z'].where(full_catalog_df.loc[:, 'Z'] > 0).isna()
full_catalog_df.loc[idx_non_Z, 'Z'] = full_catalog_df.loc[:, 'Z'].mask(idx_non_Z, full_catalog_df.loc[idx_non_Z, 'zsp'])

Create column for detection as Radio AGN

In [13]:
full_catalog_df['radio_AGN']      = (np.array(full_catalog_df.loc[:, 'is_AGN'] == 1) &\
                                     np.array(full_catalog_df.loc[:, 'LOFAR_detect'] == 1)).astype(int)

Discard minor features.

In [14]:
# full_catalog_df                     = full_catalog_df.drop(columns=['is_AGN', 'is_SDSS_QSO', 'is_SDSS_gal', 'is_gal', 'zsp'])

#### As we want to predict, only use sources that have not previous spectroscopic classification

In [15]:
print(f'The used data set (in {used_area}) has {len(full_catalog_df):,} sources.')
print(f'And {np.sum(filter_non_confirmed):,} have previous spectroscopic classification.')

The used data set (in HETDEX) has 15,136,878 sources.
And 118,734 have previous spectroscopic classification.


In [16]:
# full_catalog_df                     = full_catalog_df.loc[~filter_non_confirmed]

In [17]:
print(f'This pipeline will predict properties for {len(full_catalog_df):,} sources in {used_area}.')

This pipeline will predict properties for 15,136,878 sources in HETDEX.


---

### Load models

In [18]:
if load_models_flag:
    if use_zeroth_model:
        star_clf          = pyc.load_model(gv.models_path + gv.star_model)  # star/no-star model
        cal_star_clf      = load(gv.models_path + gv.cal_str_model)  # calibrated model
    AGN_gal_clf           = pyc.load_model(gv.models_path + gv.AGN_gal_model)  #
    cal_AGN_gal_clf       = load(gv.models_path + gv.cal_AGN_gal_model)  # calibrated model
    radio_det_AGN_clf     = pyc.load_model(gv.models_path + gv.radio_model)  # without predicted AGN
    cal_radio_det_AGN_clf = load(gv.models_path + gv.cal_radio_model)  # calibrated model
    redshift_reg_rAGN     = pyr.load_model(gv.models_path + gv.full_z_model)  # to use on full sample
    if use_second_z_model:
        redshift_reg_2    = pyr.load_model(gv.models_path + gv.high_z_model)  # sources with predicted z >= 3.6

Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded


Run predictions

In [19]:
full_catalog_df = predict_AGN_gal(full_catalog_df, 
                                  AGN_gal_clf, 
                                  cal_AGN_gal_clf, 
                                  gv.AGN_thresh, 
                                  gv.cal_AGN_thresh)

In [20]:
full_catalog_df = predict_radio_det(full_catalog_df, 
                                    radio_det_AGN_clf, 
                                    cal_radio_det_AGN_clf, 
                                    gv.radio_thresh, 
                                    gv.cal_radio_thresh)

In [21]:
full_catalog_df = full_catalog_df.rename(columns={'Score_radio': 'Score_radio_AGN', 
                                                            'pred_radio': 'pred_radio_AGN',
                                                            'Prob_radio': 'Prob_radio_AGN', 
                                                            'pred_radio_cal': 'pred_radio_cal_AGN'})

In [22]:
full_catalog_df = predict_z(full_catalog_df, redshift_reg_rAGN)

In [23]:
full_catalog_df = full_catalog_df.rename(columns={'pred_Z': 'pred_Z_rAGN'})

In [24]:
np.array(full_catalog_df.columns)

array(['RA_ICRS', 'DE_ICRS', 'Name', 'TYPE', 'Z', 'zsp', 'spCl',
       'band_num', 'LOFAR_detect', 'Sint_LOFAR', 'Sint_LOFAR_AB',
       'Speak_LOFAR', 'rms_LOFAR', 'Sint_LOFAR_non_imp',
       'Sint_LOFAR_AB_non_imp', 'Speak_LOFAR_non_imp', 'is_str',
       'is_SDSS_QSO', 'is_AGN', 'is_SDSS_gal', 'is_gal', 'W1mproPM',
       'W2mproPM', 'gmag', 'rmag', 'imag', 'zmag', 'ymag', 'W3mag',
       'W4mag', 'Jmag', 'Hmag', 'Kmag', 'g_r', 'g_i', 'g_z', 'g_y', 'g_J',
       'g_H', 'g_K', 'g_W1', 'g_W2', 'g_W3', 'g_W4', 'r_i', 'r_z', 'r_y',
       'r_J', 'r_H', 'r_K', 'r_W1', 'r_W2', 'r_W3', 'r_W4', 'i_z', 'i_y',
       'i_J', 'i_H', 'i_K', 'i_W1', 'i_W2', 'i_W3', 'i_W4', 'z_y', 'z_J',
       'z_H', 'z_K', 'z_W1', 'z_W2', 'z_W3', 'z_W4', 'y_J', 'y_H', 'y_K',
       'y_W1', 'y_W2', 'y_W3', 'y_W4', 'J_H', 'J_K', 'J_W1', 'J_W2',
       'J_W3', 'J_W4', 'H_K', 'H_W1', 'H_W2', 'H_W3', 'H_W4', 'K_W1',
       'K_W2', 'K_W3', 'K_W4', 'W1_W2', 'W1_W3', 'W1_W4', 'W2_W3',
       'W2_W4', 'W3_W4', 'class',

In [25]:
# full_catalog_df = pyc.predict_model(AGN_gal_clf, data=full_catalog_df, probability_threshold=gv.AGN_thresh, raw_score=True, round=10)
# full_catalog_df = full_catalog_df.drop(columns=['Score_0'])
# full_catalog_df = full_catalog_df.rename(columns={'Label': 'pred_class', 'Score_1': 'Score_AGN'})
# full_catalog_df['Score_AGN'] = np.around(full_catalog_df.loc[:, 'Score_AGN'], decimals=7)
# pred_probs_AGN  = cal_AGN_gal_clf.predict(full_catalog_df.loc[:, 'Score_AGN'])
# full_catalog_df['Prob_AGN'] = pred_probs_AGN

In [26]:
# full_catalog_df = pyc.predict_model(radio_det_clf, data=full_catalog_df, probability_threshold=gv.radio_thresh, raw_score=True, round=10)
# full_catalog_df = full_catalog_df.drop(columns=['Score_0'])
# full_catalog_df = full_catalog_df.rename(columns={'Label': 'pred_radio', 'Score_1': 'Score_radio'})
# full_catalog_df['Score_radio'] = np.around(full_catalog_df.loc[:, 'Score_radio'], decimals=7)
# pred_probs_rad  = cal_radio_det_clf.predict(full_catalog_df.loc[:, 'Score_radio'])
# full_catalog_df['Prob_radio'] = pred_probs_rad

In [27]:
# full_catalog_df = pyr.predict_model(redshift_reg, data=full_catalog_df, round=8)
# full_catalog_df = full_catalog_df.rename(columns={'Label': 'pred_Z'})
# full_catalog_df['pred_Z'] = np.around(full_catalog_df.loc[:, 'pred_Z'], decimals=3)

In [28]:
# redshift_tol    = 0.0
# if use_second_z_model:
#     full_catalog_df = pyr.predict_model(redshift_reg_2, data=full_catalog_df, round=6)
#     filter_pred_z   = full_catalog_df.loc[:, 'pred_Z'] >= (gv.high_z_limit + redshift_tol)
#     full_catalog_df.loc[:, 'pred_Z'] = full_catalog_df.loc[:, 'pred_Z'].mask(filter_pred_z, full_catalog_df.loc[filter_pred_z, 'Label'])
#     full_catalog_df = full_catalog_df.drop(columns=['Label'])
#     full_catalog_df.loc[:, 'pred_Z'] = np.around(full_catalog_df.loc[:, 'pred_Z'], decimals=3)

In [29]:
# full_catalog_df['pred_prob_class']    = (full_catalog_df.loc[:, 'Prob_AGN']   >= gv.cal_AGN_thresh).astype(int)
# full_catalog_df['pred_prob_radio']    = (full_catalog_df.loc[:, 'Prob_radio'] >= gv.cal_radio_thresh).astype(int)

In [30]:
full_catalog_df['pred_radio_AGN']     = (np.array(full_catalog_df.loc[:, 'pred_class'] == 1) & np.array(full_catalog_df.loc[:, 'pred_radio_AGN'] == 1)).astype(int)
full_catalog_df['Score_rAGN']         = full_catalog_df.loc[:, 'Score_AGN'] * full_catalog_df.loc[:, 'Score_radio_AGN']
full_catalog_df['pred_prob_rAGN']     = (np.array(full_catalog_df.loc[:, 'pred_class_cal'] == 1) & np.array(full_catalog_df.loc[:, 'pred_radio_cal_AGN'] == 1)).astype(int)
full_catalog_df['Prob_rAGN']          = full_catalog_df.loc[:, 'Prob_AGN'] * full_catalog_df.loc[:, 'Prob_radio_AGN']

# rad_score_scaler                      = MinMaxScaler()
# full_catalog_df['scaled_score_radio'] = rad_score_scaler.fit_transform(full_catalog_df.loc[:, 'Score_radio'].values.reshape(-1, 1))
# full_catalog_df['scaled_score_rAGN']  = full_catalog_df.loc[:, 'Score_AGN'] * full_catalog_df.loc[:, 'scaled_score_radio']

In [31]:
full_catalog_df.loc[:, ['class', 'pred_class_cal', 'LOFAR_detect', 'pred_radio_cal_AGN', 'Z', 'pred_Z_rAGN']].describe()

,class,pred_class_cal,LOFAR_detect,pred_radio_cal_AGN,Z,pred_Z_rAGN
count,118734.000000,1.513688e+07,1.513688e+07,1.513688e+07,134234.000000,1.513688e+07
mean,0.425641,6.624084e-01,1.239179e-02,9.470163e-02,0.876576,2.034189e+00
std,0.494442,4.728885e-01,1.106265e-01,2.928024e-01,0.779380,5.600387e-01
min,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,-0.004490,5.300000e-03
25%,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.381018,1.942800e+00
50%,0.000000,1.000000e+00,0.000000e+00,0.000000e+00,0.623500,2.128800e+00
75%,1.000000,1.000000e+00,0.000000e+00,0.000000e+00,1.282000,2.128800e+00
max,1.000000,1.000000e+00,1.000000e+00,1.000000e+00,7.028330,4.795500e+00


Obtain intermediate metrics

In [32]:
filter_radio_AGN_t      = np.array(full_catalog_df.loc[:, 'class'] == 1) & np.array(full_catalog_df.loc[:, 'LOFAR_detect'] == 1)
filter_gal_AGN_t        = np.array(full_catalog_df.loc[:, 'class'] == 0) | np.array(full_catalog_df.loc[:, 'class'] == 1)
total_size              = len(full_catalog_df)
filter_AGN_t            = np.array(full_catalog_df.loc[:, 'class'] == 1)
num_AGN_t               = np.sum(filter_AGN_t)
num_gal_t               = np.sum(np.array(full_catalog_df.loc[:, 'class'] == 0))
num_radio_t             = np.sum(np.array(full_catalog_df.loc[:, 'LOFAR_detect'] == 1))
num_radio_AGN_t         = np.sum(filter_radio_AGN_t)

In [33]:
# filter_radio_AGN_p      = np.array(full_catalog_df.loc[:, 'Score_AGN']          >= threshold_AGN) &\
#                             np.array(full_catalog_df.loc[:, 'Score_radio']      >= threshold_radio)
filter_radio_AGN_p      = np.array(full_catalog_df.loc[:, 'pred_class_cal']      == 1) &\
                            np.array(full_catalog_df.loc[:, 'pred_radio_cal_AGN']    == 1)
filt_hiz_rAGN_p         = filter_radio_AGN_p * np.array(full_catalog_df.loc[:, 'pred_Z_rAGN'] >= gv.high_z_limit)
filter_AGN_p            = np.array(full_catalog_df.loc[:, 'pred_class_cal']      == 1)
filter_radio_p          = np.array(full_catalog_df.loc[:, 'pred_radio_cal_AGN']      == 1)
num_AGN_p               = np.sum(filter_AGN_p)
num_gal_p               = np.sum(np.array(full_catalog_df.loc[:, 'pred_class_cal'] == 0))
num_radio_p             = np.sum(filter_radio_p)
num_radio_AGN_p         = np.sum(filter_radio_AGN_p)

### Select sources predicted to be Radio AGN (optional)

In [34]:
# full_catalog_df         = full_catalog_df.loc[filter_radio_AGN_p]

Add individual metrics for redshift

In [35]:
full_catalog_df['Delta_z_N'] = np.around((full_catalog_df.loc[:, 'pred_Z_rAGN'] - full_catalog_df.loc[:, 'Z']) /\
                            (1 + full_catalog_df.loc[:, 'Z']), decimals=3)

full_catalog_df['sigma_NMAD'] = np.around(1.48 * np.abs(full_catalog_df.loc[:, 'pred_Z_rAGN'] - full_catalog_df.loc[:, 'Z']) /\
                            (1 + full_catalog_df.loc[:, 'Z']), decimals=3)

Numerical summary.

In [36]:
str_0_t   = f'Out of {total_size:,} initial sources in {used_area},\n'
str_1_t   = f'{num_gal_t:,} are confirmed to be galaxies. On the other side,\n'
str_2_t   = f'{num_AGN_t:,} are confirmed to be AGN. And, from the AGN,\n'
str_3_t   = f'{num_radio_AGN_t:,} are detected in radio.'

str_0_p   = f'Out of {num_radio_AGN_t:,} initial radio-detected AGN in {used_area},\n'
str_1_p   = f'{num_gal_p:,} are predicted to be galaxies. On the other side,\n'
str_2_p   = f'{num_AGN_p:,} are predicted to be AGN. And, from the predicted AGN,\n'
str_3_p   = f'{num_radio_AGN_p:,} are predicted to be detected in radio.'

In [37]:
print('-' * 60)
print(str_0_t + str_1_t + str_2_t + str_3_t)
print('-' * 60)
print(str_0_p + str_1_p + str_2_p + str_3_p)
print('-' * 60)

------------------------------------------------------------
Out of 15,136,878 initial sources in HETDEX,
68,196 are confirmed to be galaxies. On the other side,
50,538 are confirmed to be AGN. And, from the AGN,
6,419 are detected in radio.
------------------------------------------------------------
Out of 6,419 initial radio-detected AGN in HETDEX,
5,110,083 are predicted to be galaxies. On the other side,
10,026,795 are predicted to be AGN. And, from the predicted AGN,
79,859 are predicted to be detected in radio.
------------------------------------------------------------


In [38]:
cols_4_table = show_columns = ['Name', 'RA_ICRS', 'DE_ICRS', 'TYPE', 'band_num', 'class', 'pred_class',
                               'pred_class_cal', 'Score_AGN', 'Prob_AGN', 'LOFAR_detect', 'pred_radio_AGN',
                               'pred_radio_cal_AGN', 'Score_radio_AGN', 'Prob_radio_AGN', 'radio_AGN', 'pred_radio_AGN',
                               'pred_prob_rAGN', 'Score_rAGN', 'Prob_rAGN', 'Z', 'pred_Z_rAGN']

In [39]:
if save_preds_flag:
    if used_area == 'S82':
        cols_2_save_preds = ['RA_ICRS', 'DE_ICRS', 'Name', 'band_num', 'class', 'Fint_VLAS82', 
                             'Fint_VLAS82_non_imp', 'W1mproPM', 'W2mproPM', 'gmag', 'rmag', 
                             'imag', 'zmag', 'ymag', 'W3mag', 'W4mag', 'Jmag', 'Hmag', 'Kmag', 
                             'Score_AGN', 'Prob_AGN', 'LOFAR_detect', 'Score_radio_AGN', 
                             'Prob_radio_AGN', 'radio_AGN', 'Score_rAGN', 'Prob_rAGN', 'Z', 
                             'pred_Z_rAGN']  # 'rms_VLAS82'
    if used_area == 'HETDEX':
        cols_2_save_preds = ['RA_ICRS', 'DE_ICRS', 'Name', 'band_num', 'class', 'Sint_LOFAR', 
                             'Sint_LOFAR_non_imp', 'W1mproPM', 'W2mproPM', 'gmag', 'rmag', 
                             'imag', 'zmag', 'ymag', 'W3mag', 'W4mag', 'Jmag', 'Hmag', 'Kmag', 
                             'Score_AGN', 'Prob_AGN', 'LOFAR_detect', 'Score_radio_AGN', 
                             'Prob_radio_AGN', 'radio_AGN', 'Score_rAGN', 'Prob_rAGN', 'Z', 'pred_Z_rAGN']

In [40]:
cols_4_export_S82    = ['Fint_VLAS82']
cols_4_export_HETDEX = ['Sint_LOFAR']
cols_4_export_COSMOS = ['Flux_COSMOSVLA3']

cols_4_exp_all       = {'S82': cols_4_export_S82, 'HETDEX': cols_4_export_HETDEX, 'COSMOS': cols_4_export_COSMOS}

cols_photo           = ['W1mproPM', 'W2mproPM', 'gmag', 'rmag', 'imag', 'zmag', 
                        'ymag', 'Jmag', 'Hmag', 'Kmag', 'W3mag', 'W4mag']

cols_4_export        = cols_4_table + cols_4_exp_all[used_area] + cols_photo

In [41]:
filter_new_rAGN = full_catalog_df.loc[:, 'radio_AGN'] == 0
if predict_only_hi_z:
    filter_high_z   = full_catalog_df.loc[:, 'pred_Z_rAGN']    >= gv.high_z_limit
elif not predict_only_hi_z:
    filter_high_z   = np.ones_like(full_catalog_df.loc[:, 'pred_Z_rAGN']).astype(bool)

In [42]:
full_catalog_df.loc[:, 'TYPE'] = full_catalog_df.loc[:, 'TYPE'].mask(full_catalog_df.loc[:, 'TYPE'] == '    ', 'CCCC')

In [43]:
display(full_catalog_df.loc[filter_high_z, cols_4_export].sort_values(by=['pred_Z_rAGN'], ascending=False).head(15))

,Name,RA_ICRS,DE_ICRS,TYPE,band_num,class,pred_class,pred_class_cal,Score_AGN,Prob_AGN,LOFAR_detect,pred_radio_AGN,pred_radio_cal_AGN,Score_radio_AGN,Prob_radio_AGN,radio_AGN,pred_radio_AGN,pred_prob_rAGN,Score_rAGN,Prob_rAGN,Z,pred_Z_rAGN,Sint_LOFAR,W1mproPM,W2mproPM,gmag,rmag,imag,zmag,ymag,Jmag,Hmag,Kmag,W3mag,W4mag
7355795,J133422.63+475033.4,203.594269,47.842628,QX,8,1.0,1,1,0.500056,0.892290,1,0,0,0.048814,0.062214,1,0,0,0.024410,0.055513,4.958,4.7955,5.400000,18.983999,19.130001,23.299999,20.933100,19.620399,19.306101,19.200300,17.450001,17.24,16.59,16.67,14.62
3226938,J111714.13+540919.4,169.308868,54.155396,Q,8,1.0,1,1,0.500068,0.926060,1,0,1,0.208025,0.225272,1,0,1,0.104026,0.208615,4.700,4.7479,0.600000,19.456999,19.754002,23.299999,21.331800,20.118099,19.834101,19.797501,17.450001,17.24,16.59,16.67,14.62
4264628,J141000.75+460134.5,212.503098,46.026260,,6,NaN,0,0,0.499961,0.204968,0,0,1,0.248813,0.261320,0,0,0,0.124397,0.053562,NaN,4.7153,0.356451,19.681999,19.809999,23.299999,20.781799,19.471001,19.164101,19.027300,17.450001,17.24,16.59,16.67,14.62
9544254,J132514.21+534447.1,201.309235,53.746429,,6,NaN,0,1,0.500007,0.579070,0,0,1,0.351672,0.345250,0,0,1,0.175838,0.199924,NaN,4.7114,0.356451,18.823999,19.125999,23.299999,20.935801,19.635599,19.379000,19.478600,17.450001,17.24,16.59,16.67,14.62
10831188,J124400.05+553406.8,191.000183,55.568558,Q,8,1.0,1,1,0.500053,0.880950,0,0,1,0.736330,0.616554,0,0,1,0.368204,0.543153,4.623,4.6540,0.356451,18.931999,19.226000,23.299999,21.077101,19.595501,19.544100,19.271700,17.450001,17.24,16.59,16.67,14.62
13292816,J143508.80+485521.4,218.786652,48.922630,Q,8,1.0,1,1,0.500043,0.833659,0,0,0,0.131533,0.152147,0,0,0,0.065772,0.126839,4.776,4.6515,0.356451,19.424000,19.650002,23.299999,21.011400,19.706301,19.509300,19.394300,17.450001,17.24,16.59,16.67,14.62
12355845,J144321.14+501908.4,220.838120,50.319016,,5,NaN,0,1,0.500007,0.579070,0,0,1,0.937123,0.794128,0,0,1,0.468568,0.459856,NaN,4.6064,0.356451,19.511999,19.809999,23.299999,21.092501,19.449699,19.490801,21.400000,17.450001,17.24,16.59,16.67,14.62
13669337,J143442.05+532102.5,218.675201,53.350716,,7,NaN,0,0,0.499916,0.046814,0,0,1,0.438486,0.410208,0,0,0,0.219206,0.019203,NaN,4.5943,0.356451,19.473999,19.809999,23.299999,20.897600,20.666800,20.399200,21.400000,17.450001,17.24,16.59,16.67,14.62
3108302,J111513.19+532105.5,168.804932,53.351532,,7,NaN,0,0,0.499973,0.282964,0,0,0,0.089812,0.108494,0,0,0,0.044904,0.030700,NaN,4.5863,0.356451,19.511000,19.809999,23.299999,20.940901,20.626600,20.388500,21.400000,17.450001,17.24,16.59,16.67,14.62
5398215,J140947.84+514530.7,212.449341,51.758556,,5,NaN,0,0,0.499945,0.123539,0,0,1,0.305554,0.308715,0,0,0,0.152760,0.038138,NaN,4.5842,0.356451,19.552999,19.809999,23.299999,21.259800,20.958599,20.683599,21.400000,17.450001,17.24,16.59,16.67,14.62


In [44]:
if save_preds_flag:
    saving_data_pred       = full_catalog_df.loc[:, cols_2_save_preds]
    # full_catalog_df.loc[filter_high_z, cols_4_export].sort_values(by=['pred_Z_rAGN'],
    #  ascending=False).to_csv(gv.preds_path + f'predicted_rAGN_{used_area}.csv', index_label='ID')
    file_name_csv  = gv.preds_path + f'predicted_rAGN_{used_area}.csv'
    file_name_prqt = gv.preds_path + f'predicted_rAGN_{used_area}.parquet'
    saving_data_pred.to_csv(file_name_csv, index_label='ID')
    # saving_data_pred['ID'] = saving_data_pred.index
    saving_data_pred.to_parquet(file_name_prqt, index=True, engine='fastparquet')
    print(f'Data from {used_area} saved in files {file_name_csv} and {file_name_prqt}.')
    print(f'Each file contains {len(saving_data_pred)} entries.')

Data from HETDEX saved in files pred_rAGN/predicted_rAGN_HETDEX.csv and pred_rAGN/predicted_rAGN_HETDEX.parquet.
Each file contains 15136878 entries.


---

In [45]:
temp_flag = False

In [46]:
if temp_flag:
    cols_AGN   = ['g_r', 'r_i', 'r_J', 'i_z', 'i_y', 
                  'z_y', 'z_W2', 'y_J', 'y_W1', 'y_W2', 'J_H', 'H_K', 
                  'H_W3', 'W1_W2', 'W1_W3', 'W3_W4']  # Only colours (no 'band_num', 'W4mag')
    cols_radio = ['g_r', 'g_i', 'r_i', 'r_z', 'i_z', 
                  'z_y', 'z_W1', 'y_J', 'y_W1', 'J_H', 'H_K', 'K_W3', 
                  'K_W4', 'W1_W2', 'W2_W3']  # Only colours (no 'band_num', 'W4mag')
    cols_z     = ['g_r', 'g_W3', 'r_i', 'r_z', 'i_z', 
                  'i_y', 'z_y', 'y_J', 'y_W1', 'J_H', 'H_K', 'K_W3', 
                  'K_W4', 'W1_W2', 'W2_W3']  # Only colours (no 'band_num', 'W4mag')
    
    cols_cols = list(np.unique(cols_AGN + cols_radio + cols_z))

In [47]:
if temp_flag:
    
    add_columns  = ['band_num', 'class', 'pred_class_cal', 'Score_AGN', 'Prob_AGN', 
                    'LOFAR_detect', 'pred_radio_cal_AGN', 'Score_radio_AGN', 'Prob_radio_AGN', 
                    'radio_AGN', 'pred_prob_rAGN', 'Score_rAGN', 'Prob_rAGN', 'Z', 'pred_Z_rAGN']
    used_colours = cols_cols

In [48]:
if temp_flag:
    saving_data = full_catalog_df.loc[:, add_columns + cols_photo + used_colours]
    saving_data['ID'] = saving_data.index
    file_name         = gv.preds_path + f'{used_area}_for_prediction.h5'
    file_name_pqt     = gv.preds_path + f'{used_area}_for_prediction.parquet'
    saving_data.to_hdf(file_name, key='df')
    saving_data.to_parquet(file_name_pqt, index=True, engine='fastparquet')
    print(f'Data from {used_area} saved in files {file_name} and {file_name_pqt}')

In [49]:
save_full_flag = False

In [50]:
np.array(full_catalog_df.columns)

array(['RA_ICRS', 'DE_ICRS', 'Name', 'TYPE', 'Z', 'zsp', 'spCl',
       'band_num', 'LOFAR_detect', 'Sint_LOFAR', 'Sint_LOFAR_AB',
       'Speak_LOFAR', 'rms_LOFAR', 'Sint_LOFAR_non_imp',
       'Sint_LOFAR_AB_non_imp', 'Speak_LOFAR_non_imp', 'is_str',
       'is_SDSS_QSO', 'is_AGN', 'is_SDSS_gal', 'is_gal', 'W1mproPM',
       'W2mproPM', 'gmag', 'rmag', 'imag', 'zmag', 'ymag', 'W3mag',
       'W4mag', 'Jmag', 'Hmag', 'Kmag', 'g_r', 'g_i', 'g_z', 'g_y', 'g_J',
       'g_H', 'g_K', 'g_W1', 'g_W2', 'g_W3', 'g_W4', 'r_i', 'r_z', 'r_y',
       'r_J', 'r_H', 'r_K', 'r_W1', 'r_W2', 'r_W3', 'r_W4', 'i_z', 'i_y',
       'i_J', 'i_H', 'i_K', 'i_W1', 'i_W2', 'i_W3', 'i_W4', 'z_y', 'z_J',
       'z_H', 'z_K', 'z_W1', 'z_W2', 'z_W3', 'z_W4', 'y_J', 'y_H', 'y_K',
       'y_W1', 'y_W2', 'y_W3', 'y_W4', 'J_H', 'J_K', 'J_W1', 'J_W2',
       'J_W3', 'J_W4', 'H_K', 'H_W1', 'H_W2', 'H_W3', 'H_W4', 'K_W1',
       'K_W2', 'K_W3', 'K_W4', 'W1_W2', 'W1_W3', 'W1_W4', 'W2_W3',
       'W2_W4', 'W3_W4', 'class',

In [51]:
if save_full_flag:
    if used_area == 'S82':
        cols_2_save = ['RA_ICRS', 'DE_ICRS', 'Name', 'TYPE', 'Z', 'band_num', 'Fint_VLAS82', 
                       'Fint_VLAS82_AB', 'Fint_VLAS82_non_imp', 'is_str', 'W1mproPM', 
                       'W2mproPM', 'gmag', 'rmag', 'imag', 'zmag', 'ymag', 'W3mag', 'W4mag', 'Jmag', 
                       'Hmag', 'Kmag', 'g_r', 'g_i', 'g_z', 'g_y', 'g_J', 'g_H', 'g_K', 'g_W1', 
                       'g_W2', 'g_W3', 'g_W4', 'r_i', 'r_z', 'r_y', 'r_J', 'r_H', 'r_K', 'r_W1', 
                       'r_W2', 'r_W3', 'r_W4', 'i_z', 'i_y', 'i_J', 'i_H', 'i_K', 'i_W1', 'i_W2', 
                       'i_W3', 'i_W4', 'z_y', 'z_J', 'z_H', 'z_K', 'z_W1', 'z_W2', 'z_W3', 'z_W4', 
                       'y_J', 'y_H', 'y_K', 'y_W1', 'y_W2', 'y_W3', 'y_W4', 'J_H', 'J_K', 'J_W1', 
                       'J_W2', 'J_W3', 'J_W4', 'H_K', 'H_W1', 'H_W2', 'H_W3', 'H_W4', 'K_W1', 
                       'K_W2', 'K_W3', 'K_W4', 'W1_W2', 'W1_W3', 'W1_W4', 'W2_W3', 'W2_W4', 'W3_W4', 
                       'LOFAR_detect', 'class', 'radio_AGN', 'Prob_AGN', 'Prob_radio_AGN', 
                       'pred_Z_rAGN', 'pred_class_cal', 'pred_radio_cal_AGN', 'pred_prob_rAGN', 'Prob_rAGN']  # 'rms_VLAS82'
    if used_area == 'HETDEX':
        cols_2_save = ['RA_ICRS', 'DE_ICRS', 'Name', 'TYPE', 'Z', 'band_num', 'Sint_LOFAR', 
                       'Sint_LOFAR_AB', 'Sint_LOFAR_non_imp', 'rms_LOFAR', 'Speak_LOFAR', 
                       'Speak_LOFAR_non_imp', 'is_str', 'W1mproPM', 'W2mproPM', 'gmag', 'rmag', 
                       'imag', 'zmag', 'ymag', 'W3mag', 'W4mag', 'Jmag', 'Hmag', 'Kmag', 'g_r', 
                       'g_i', 'g_z', 'g_y', 'g_J', 'g_H', 'g_K', 'g_W1', 'g_W2', 'g_W3', 'g_W4', 
                       'r_i', 'r_z', 'r_y', 'r_J', 'r_H', 'r_K', 'r_W1', 'r_W2', 'r_W3', 'r_W4', 
                       'i_z','i_y', 'i_J', 'i_H', 'i_K', 'i_W1', 'i_W2', 'i_W3', 'i_W4', 'z_y', 
                       'z_J', 'z_H', 'z_K', 'z_W1', 'z_W2', 'z_W3', 'z_W4', 'y_J', 'y_H', 'y_K', 
                       'y_W1', 'y_W2', 'y_W3', 'y_W4', 'J_H', 'J_K', 'J_W1', 'J_W2', 'J_W3', 'J_W4', 
                       'H_K', 'H_W1', 'H_W2', 'H_W3', 'H_W4', 'K_W1', 'K_W2', 'K_W3', 'K_W4', 
                       'W1_W2', 'W1_W3', 'W1_W4', 'W2_W3', 'W2_W4', 'W3_W4', 'LOFAR_detect', 
                       'class', 'radio_AGN', 'Prob_AGN', 'Prob_radio_AGN', 'pred_Z_rAGN', 
                       'pred_class_cal', 'pred_radio_cal_AGN', 'pred_prob_rAGN', 'Prob_rAGN']

In [52]:
if save_full_flag:
    saving_data_full       = full_catalog_df.loc[:, cols_2_save]
    saving_data_full['ID'] = saving_data_full.index
    try:
        saving_data_full.to_hdf(gv.preds_path + f'{used_area}_full_prediction.h5', key='df')
        print(f'File {gv.preds_path}{used_area}_full_prediction.h5 saved')
    except:
        print(f'File {gv.preds_path}{used_area}_full_prediction.h5 was not saved')
    try:
        saving_data_full.to_parquet(gv.preds_path + f'{used_area}_full_prediction.parquet', index=True, engine='fastparquet')
        print(f'File {gv.preds_path}{used_area}_full_prediction.parquet saved')
    except:
        print(f'File {gv.preds_path}{used_area}_full_prediction.parquet was not saved')

File pred_rAGN/HETDEX_full_prediction.h5 saved
File pred_rAGN/HETDEX_full_prediction.parquet saved
